In [ ]:
# Melakukan import dataset standard iris dan memisahkan data dan target
from sklearn import datasets
iris_dataset = datasets.load_iris()
X_iris = iris_dataset.data
Y_iris = iris_dataset.target
print(X_iris[0:5])
print(Y_iris[0:5])

In [ ]:
# Melakukan import dataset csv play tennis dan memisahkan data dan target
import pandas
ori_data_frame = pandas.read_csv('tennis.csv')

# axis = 1 artinya mau drop column
data_frame = ori_data_frame.drop('play', axis=1)
data_frame = data_frame.select_dtypes(include=[object])
# print(data_frame.columns)

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df_2 = data_frame.apply(le.fit_transform)
# print(df_2)

one_hot_encoder = preprocessing.OneHotEncoder()
one_hot_encoder.fit(df_2)
one_hot_labels = one_hot_encoder.transform(df_2).toarray()

X_tennis = one_hot_labels
Y_tennis = []

for row in ori_data_frame.index:
    Y_tennis.append(ori_data_frame['play'][row])

print(X_tennis[0:5])
print(Y_tennis[0:5])

In [ ]:
# Melakukan pembelajaran full training dengan DecisionTree
from sklearn import tree
clf_tree = tree.DecisionTreeClassifier()
clf_tree = clf_tree.fit(X_iris, Y_iris)
clf_tree_2 = tree.DecisionTreeClassifier()
clf_tree_2 = clf_tree_2.fit(X_tennis, Y_tennis)
# Menampilkan hasil prediksi terhadap 5 instance awal
print(clf_tree.predict(X_iris[:5]))
print(clf_tree_2.predict(X_tennis[:5]))

In [ ]:
# Menampilkan model pembelajaran tree
data_feature_names = iris_dataset.feature_names
data_feature_names_2 = ['outlook', 'temp', 'humidity']
data_target_names = iris_dataset.target_names
data_target_names_2 = ['no', 'yes']
dot_model = tree.export_graphviz(clf_tree, feature_names = data_feature_names, class_names = data_target_names, out_file = None) 
dot_model_2 = tree.export_graphviz(clf_tree_2, class_names = data_target_names_2, out_file = None) 

# import graphviz
# import os
# os.environ["PATH"] += os.pathsep + 'D:/Graphviz2.38/bin/'
# tree_viz = graphviz.Source(dot_model) 
# tree_viz.render('tree_iris')
# tree_viz_2 = graphviz.Source(dot_model_2) 
# tree_viz_2.render('tree_tennis')

In [ ]:
# Melakukan pembelajaran full training dengan MLP
from sklearn.neural_network import MLPClassifier
clf_mlp = MLPClassifier(solver='sgd', max_iter = 10000, alpha=1e-3, hidden_layer_sizes=(4,3))
clf_mlp.fit(X_iris, Y_iris)
clf_mlp_2 = MLPClassifier(solver='sgd', max_iter = 10000, alpha=1e-3, hidden_layer_sizes=(4,3))
clf_mlp_2.fit(X_tennis, Y_tennis)
# Menampilkan hasil prediksi terhadap 5 instance awal
print(clf_mlp.predict(X_iris[:5]))
print(clf_mlp_2.predict(X_tennis[:5]))

In [ ]:
# Menampilkan model pembelajaran MLP untuk iris
for i in range(0, len(clf_mlp.coefs_)):
    print('Intercepts:')
    print(clf_mlp.intercepts_[i])
    print('Weights:')
    print(clf_mlp.coefs_[i])

In [ ]:
# Menampilkan model pembelajaran MLP untuk tennis
for i in range(0, len(clf_mlp_2.coefs_)):
    print('Intercepts:')
    print(clf_mlp_2.intercepts_[i])
    print('Weights:')
    print(clf_mlp_2.coefs_[i])

In [ ]:
# Melakukan import dan export model
from sklearn.externals import joblib
def save_model(model, filename):
    joblib.dump(model, filename)
    
def load_model(filename):
    clf = joblib.load(filename)
    return clf

In [ ]:
# Meng-export full training MLP dan DTree
save_model(clf_tree, "full_training_Tree_Iris.model")
save_model(clf_mlp, "full_training_MLP_Iris.model")
save_model(clf_tree_2, "full_training_Tree_Tennis.model")
save_model(clf_mlp_2, "full_training_MLP_Tennis.model")

# Me-load full training MLP dan DTree
clf_mlp = load_model("full_training_MLP_Iris.model")
clf_tree = load_model("full_training_Tree_Iris.model")
clf_mlp_2 = load_model("full_training_MLP_Tennis.model")
clf_tree_2 = load_model("full_training_Tree_Tennis.model")

# Tes apakah berhasil load dengan melakukan prediksi
print(clf_mlp.predict(X_iris[:5]))
print(clf_tree.predict(X_iris[:5]))
print(clf_mlp_2.predict(X_tennis[:5]))
print(clf_tree_2.predict(X_tennis[:5]))

In [ ]:
#C ANN + Decision Tree Split test
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix

#Pisah data menjadi train dan test
irisTrainX, irisTestX, irisTrainY, irisTestY =  train_test_split(X_iris, Y_iris, test_size=0.1, train_size=0.9)

#Make Classifier
decisionTree = tree.DecisionTreeClassifier()
multiLayerANN = MLPClassifier(solver = "lbfgs")
decisionTree.fit(irisTrainX, irisTrainY)
multiLayerANN.fit(irisTrainX, irisTrainY)

treePredict = decisionTree.predict(irisTestX)
annPredict = multiLayerANN.predict(irisTestX)

print(treePredict)
treeAccuracy = accuracy_score(irisTestY, treePredict)
treeMatrix = confusion_matrix(irisTestY, treePredict)
print()
print("Accuracy : ") 
print(treeAccuracy)
print()
print("Confusion Matrix :")
print(treeMatrix)
print()

print(annPredict)
annAccuracy = accuracy_score(irisTestY, annPredict)
annMatrix = confusion_matrix(irisTestY, annPredict)
print()
print("Accuracy : ") 
print(annAccuracy)
print()
print("Confusion Matrix :")
print(annMatrix)
print()




In [ ]:
#D 10-fold Cross validation for decision tree and ANN
from sklearn.model_selection import cross_val_score
decisionTree = tree.DecisionTreeClassifier()
multiLayerANN = MLPClassifier(solver = "lbfgs")

treeCrossVal = cross_val_score(decisionTree, X_iris, Y_iris, cv=10)
print("\nTree Score :")
print(treeCrossVal)
print("Accuracy: %0.2f (+/- %0.2f)" % (treeCrossVal.mean(), treeCrossVal.std() * 2))
annCrossVal = cross_val_score(multiLayerANN, X_iris, Y_iris, cv=10)
print("\nANN Score :")
print(annCrossVal)
print("Accuracy: %0.2f (+/- %0.2f)" % (annCrossVal.mean(), annCrossVal.std() * 2))



In [4]:
#G Create new instances
from sklearn import datasets
import numpy as np

iris_dataset = datasets.load_iris()
X_iris = iris_dataset.data
Y_iris = iris_dataset.target

X = []
print("Input attributes with", X_iris.dtype, " type") 
for i in range(0, len(X_iris[0])):
    inp = input()
    X.append(inp)

X = np.array(X, dtype=X_iris.dtype)
print("Input attributes with", Y_iris.dtype, " type") 
Y = np.array(input(), dtype=Y_iris.dtype)

print("\nX = ", X)
print("Y = ", Y)

Input attributes with float64  type
1.2
2.3
4.1
2.3
Input attributes with int64  type
5

X =  [ 1.2  2.3  4.1  2.3]
Y =  5
